## Physics based MCQ Generator

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
curpath = os.getcwd()
os.chdir(curpath.split("core")[0])

In [3]:
import openai
import json

from prompts import SIMPLE_PHYSICS_NUMERICAL_MCQ_PROMPT, SIMPLE_PHYSICS_TEXT_MCQ_PROMPT, CALCULATION_CHECK_PROMPT
from core.features.qna_gen.helper import get_results_from_wolfram_cloud
from core.features.qna_gen.utils import add_dicts

from dotenv import load_dotenv
load_dotenv()

True

### Text MCQs

In [4]:

def generate_text_mcqs(context, n=1, example = None, model_id = "gpt-4-0613"):

    system_message = {"role": "system", "content": SIMPLE_PHYSICS_TEXT_MCQ_PROMPT}
    conversation = [system_message]

    global total_usage
    total_usage = {'prompt_tokens': 0, 'completion_tokens': 0, 'total_tokens': 0}

    user_prompt = f"""
    CONTEXT_TEXT:
    //context//

    {context}

    //context//
   
    N: {n}

    EXAMPLE:
    {example if example else "NA"}

    QUESTIONS: 
    """

    user_message = {'role': 'user', "content": user_prompt}
    conversation.append(user_message)

    response = openai.ChatCompletion.create(
                                    model= model_id,
                                    messages = conversation,
                                    temperature = 1
                                    )
    
    total_usage = add_dicts(total_usage, dict(response.usage))

    output = response.choices[0].message.content
    
    return {"output": output, "total_usage": total_usage}

context = """
Thermodynamics

Definitions:
1. System: A part of the universe under study.
2. Surroundings: Everything other than the system.
3. Thermodynamic State: The description of a system in terms of properties like pressure, volume, temperature, etc.
4. State Functions: Properties that depend only on the state of the system, not the path. Examples: U (Internal energy), H (Enthalpy), S (Entropy), and G (Gibbs free energy).
5. Path Functions: Depend on the path taken during a change, like heat (q) and work (w).
6. First Law of Thermodynamics: ΔU = q + w, where ΔU is the change in internal energy, q is heat, and w is work.
7. Isothermal Process: Process in which temperature remains constant.
8. Adiabatic Process: No heat exchange with surroundings. 

Formulas:
1. w = -P_ext x ΔV 
   - Work done during a quasi-static isothermal expansion/compression.
2. For an ideal gas undergoing isothermal expansion or compression:
   w = -nRT ln(Vf/Vi)
3. For an adiabatic process in an ideal gas:
   PV^γ = constant
   where γ is the ratio of specific heats.

"""

example = {
        "question": "Which of the following statements is true regarding thermodynamics?",
        "choices": ["The surroundings refer to the part of the universe under study.", "State Functions depend on the path taken during a change.", "In an adiabatic process, there is no heat exchange with the surroundings.", "The First Law of Thermodynamics is represented by the equation ΔU = q - w."],
        "answer": "In an adiabatic process, there is no heat exchange with the surroundings."
    }

output = generate_text_mcqs(
                        context = context,
                        n = 5,
                        # example = example,
                        )

print(output['output'])

{'output': '\n{\n    "mcqs": [\n        {\n            "question": "What are the properties described by State Functions?",\n            "choices": {"a": "They depend on the path taken during a change.", "b": "They depend only on the state of the system, not the path.", "c": "They describe the work done on the system.", "d": "They are not dependent on any internal properties of the system."},\n            "answer": {"b": "They depend only on the state of the system, not the path."}\n        },\n        {\n            "question": "Which of the following is true about an adiabatic process?",\n            "choices": {"a": "The temperature remains constant.", "b": "There is a heat exchange with surroundings.", "c": "There is no heat exchange with surroundings.", "d": "The pressure of the system increases."},\n            "answer": {"c": "There is no heat exchange with surroundings."}\n        },\n        {\n            "question": "What is the First Law of Thermodynamics?",\n            "c

KeyError: 0

In [19]:
print(output[0])

{
    "mcq": [
        {
            "question": "What is a system in thermodynamics?",
            "choices": ["A subsystem of a larger thermodynamic construct.", "Everything that is not part of the surroundings.", "Particles that interact with each other.", "A part of the universe under study."],
            "answer": "A part of the universe under study."
        },
        {
            "question": "What are state functions and path functions in thermodynamics?",
            "choices": ["Examples of State functions are U, H, S, G and path functions are q, w.", "State functions are the ones that depend only on the path of the process and path functions depend only on the state.", "State functions describe the path of a process, while path functions describe the state of a system.", "State functions depend on the state of the system and are independent of the path of the process while the path functions depend on the path of the process."],
            "answer": "State functions depen

### Numerical MCQs

In [ ]:
json_ques = json.loads(output['output'])

str(list(json_ques.values())[0][0]).replace("'", '"')

In [13]:
functions = [
    {
      "name": "get_results_from_wolfram_alpha",
      "description": """This function uses wolfram alpha endpoint that Understands natural language queries about entities in chemistry, physics, geography, history, art, astronomy, and more.\n- Performs mathematical calculations, date and unit conversions, formula solving, etc.\n- Convert inputs to simplified keyword queries whenever possible (e.g. convert \"how many people live in France\" to \"France population\").\n- Use ONLY single-letter variable names, with or without integer subscript (e.g., n, n1, n_1).\n- Use named physical constants (e.g., 'speed of light') without numerical substitution.\n- Include a space between compound units (e.g., \"Ω m\" for \"ohm*meter\").\n- To solve for a variable in an equation with units, consider solving a corresponding equation without units; exclude counting units (e.g., books), include genuine units (e.g., kg).""",
      "parameters": {
        "type": "object",
        "properties": {
          "queries": {
            "type": "string",
            "description": """curated natural language queries for wolfram alpha
            for example: "distance from earth to mars; current population of new delhi"."""
          }
        }
      },
        "required": ["queries"]
    },
    {
      "name": "get_results_from_wolfram_cloud",
      "description": """Use this function for problems solvable with Wolfram Language code.
      
      """,
      "parameters": {
        "type": "object",
        "properties": {
          "queries": {
            "type": "string",
            "description": """the input wolfram language query for mathematical calculations in string. ONLY proper wolfram language queries are supported.
            Format for Query:-
            "a = Solve[aCoeff1*aVar == Var1 - Offset1 && aCoeff2*aVar == Var1 + Offset2, {aVar, Var1}][[1, 1, 2]]".
            """
          }
        }
      },
        "required": ["queries"]
    },
] 

In [8]:

def generate_numerical_mcqs(context, n=1, example = None, model_id = "gpt-4-0613", return_usage = False):

    system_message = {"role": "system", "content": SIMPLE_PHYSICS_NUMERICAL_MCQ_PROMPT}
    conversation = [system_message]

    global total_usage
    total_usage = {'prompt_tokens': 0, 'completion_tokens': 0, 'total_tokens': 0}

    user_prompt = f"""
    CONTEXT_TEXT:
    //context//

    {context}

    //context//
   
    N: {n}

    EXAMPLE:
    {example if example else "NA"}

    QUESTIONS: 
    """

    user_message = {'role': 'user', "content": user_prompt}
    conversation.append(user_message)

    response = openai.ChatCompletion.create(
                                    model= model_id,
                                    messages = conversation,
                                    temperature = 1
                                    )
    
    total_usage = add_dicts(total_usage, dict(response.usage))

    output = response.choices[0].message.content

    if return_usage:
        return output, total_usage
    
    return output

context = """
Thermodynamics

Definitions:
1. System: A part of the universe under study.
2. Surroundings: Everything other than the system.
3. Thermodynamic State: The description of a system in terms of properties like pressure, volume, temperature, etc.
4. State Functions: Properties that depend only on the state of the system, not the path. Examples: U (Internal energy), H (Enthalpy), S (Entropy), and G (Gibbs free energy).
5. Path Functions: Depend on the path taken during a change, like heat (q) and work (w).
6. First Law of Thermodynamics: ΔU = q + w, where ΔU is the change in internal energy, q is heat, and w is work.
7. Isothermal Process: Process in which temperature remains constant.
8. Adiabatic Process: No heat exchange with surroundings. 

Formulas:
1. w = -P_ext x ΔV 
   - Work done during a quasi-static isothermal expansion/compression.
2. For an ideal gas undergoing isothermal expansion or compression:
   w = -nRT ln(Vf/Vi)
3. For an adiabatic process in an ideal gas:
   PV^γ = constant
   where γ is the ratio of specific heats.

"""

example = {
        "q": "An ideal gas undergoes an isothermal expansion. If the initial volume Vi is 2 liters and the final volume Vf is 5 liters, the number of moles n is 3, the gas constant R is 8.314 J/mol·K, and the temperature T is 300 K, calculate the work done during this process.",
        "c": ["-5480.26 J", "-6856.24 J", "-7200.00 J", "-6500.00 J"],
        "a": "-6856.24 J"
    }

output = generate_numerical_mcqs(
                        context = context,
                        n = 5,
                        # example = example,
                        return_usage=True
                        )

print(output)

('\n    {\n        "mcqs": [\n            {\n                "q": "If a system expands against a constant external pressure and the volume changes from 4 L to 6 L with a P_ext of 1 atm, how much work is done by the system?",\n                "c": ["-2 J", "-2 atm.L", "2 atm.L", "2 J"],\n                "a": "-2 atm.L"\n            },\n            {\n                "q": "For a gas undergoing isothermal expansion where n = 2 moles, R=8.314 J/(K.mol), T=298 K and the volume changes from 10 L to 20 L. What is the work done by the gas?",\n                "c": ["-3315.94 J", "-6615.94 J", "6615.94 J", "3315.94 J"],\n                "a": "-3315.94 J"\n            },\n            {\n                "q": "If a gas undergoes an adiabatic process and the initial pressure and volume are 5 atm and 10 L respectively. The volume then changes to 20 L. What would be the final pressure if γ = 1.4?",\n                "c": ["2.55 atm", "5.12 atm", "10.24 atm", "20.48 atm"],\n                "a": "2.55 at

In [9]:
print(output[0])


    {
        "mcqs": [
            {
                "q": "If a system expands against a constant external pressure and the volume changes from 4 L to 6 L with a P_ext of 1 atm, how much work is done by the system?",
                "c": ["-2 J", "-2 atm.L", "2 atm.L", "2 J"],
                "a": "-2 atm.L"
            },
            {
                "q": "For a gas undergoing isothermal expansion where n = 2 moles, R=8.314 J/(K.mol), T=298 K and the volume changes from 10 L to 20 L. What is the work done by the gas?",
                "c": ["-3315.94 J", "-6615.94 J", "6615.94 J", "3315.94 J"],
                "a": "-3315.94 J"
            },
            {
                "q": "If a gas undergoes an adiabatic process and the initial pressure and volume are 5 atm and 10 L respectively. The volume then changes to 20 L. What would be the final pressure if γ = 1.4?",
                "c": ["2.55 atm", "5.12 atm", "10.24 atm", "20.48 atm"],
                "a": "2.55 atm"
            },


In [14]:
# calculation check and improvements

def calculation_checking(input, model_id = "gpt-4-0613"):

    total_usage = {'prompt_tokens': 0, 'completion_tokens': 0, 'total_tokens': 0}
    
    input_prompt = f"""

    INPUT: 

    {input}

    OUTPUT: 

    """ 
    
    system_message = {"role": "system", "content": CALCULATION_CHECK_PROMPT}
    conversation = [system_message]
    input_message = {'role': 'user', 'content': input_prompt}
    conversation.append(input_message)

    response = openai.ChatCompletion.create(
                                    model= model_id,
                                    messages = conversation,
                                    functions = functions,
                                    function_call = {'name': "get_results_from_wolfram_cloud"},
                                    temperature = 0
                                )

    total_usage = add_dicts(total_usage, dict(response.usage))
    print()
    print("AFTER FUNCTION", total_usage)
    print("calling function".upper())
    function_call = response.choices[0]['message']['function_call']
    conversation.append(response.choices[0]['message'])

    # print(response)

    # if function_call["name"] == "get_results_from_wolfram_cloud":
    queries = json.loads(function_call["arguments"])["queries"]
    print("QUERIES", queries)

    try:
        print("good queries", queries)
        result = get_results_from_wolfram_cloud(queries)
    except:
        raise Exception("error from wolfram")

# else:
#     raise "some error in function name"

    print("result from function".upper(), str(result))
    conversation.append({'role': 'function', "name": function_call["name"], "content": str(result)})

    response = openai.ChatCompletion.create(
        model = model_id,
        messages = conversation,
        functions = functions,
        temperature = 0.2
    )

    # print(response)

    # conversation.append(response.choices[0].message)
    total_usage = add_dicts(total_usage, dict(response.usage))
    print(total_usage)
    # return response, conversation
    
    # response, conversation = response_with_function(response, conversation)
    output = response.choices[0].message.content

    return {"output": output, "total_usage": total_usage}

# input = output['output']

correct_ques = []
total_usage = {'prompt_tokens': 0, 'completion_tokens': 0, 'total_tokens': 0}
for ques in list(json_ques.values())[0]:
    ques = str(ques).replace("'", '"')
    result = calculation_checking(ques)
    total_usage = add_dicts(total_usage, result["total_usage"])
    correct_ques.append(result["output"])

print(correct_ques)

CALLING FUNCTION
QUERIES ['1 atm * (6 L - 4 L)', '2 moles * 8.314 J/(K.mol) * 298 K * ln(20 L/10 L)', '5 atm * (10 L)^1.4 / (20 L)^1.4']
good queries ["['1 atm * (6 L - 4 L)'", " '2 moles * 8.314 J/(K.mol) * 298 K * ln(20 L/10 L)'", " '5 atm * (10 L)^1.4 / (20 L)^1.4']"]
RESULT FROM FUNCTION {"['1 atm * (6 L - 4 L)'": '18.83 kg m^4/s^2 (kilogram meters to the fourth per second squared)', " '2 moles * 8.314 J/(K.mol) * 298 K * ln(20 L/10 L)'": '3434.6 K J/к (kelvin joules per Russian kopek)', " '5 atm * (10 L)^1.4 / (20 L)^1.4']": '17835 kg m/s^2 (kilogram meters per second squared)'}

    {
        "mcqs": [
            {
                "q": "If a system expands against a constant external pressure and the volume changes from 4 L to 6 L with a P_ext of 1 atm, how much work is done by the system?",
                "c": ["-2 J", "-2 atm.L", "2 atm.L", "2 J"],
                "a": "-2 atm.L"
            },
            {
                "q": "For a gas undergoing isothermal expansion wher